In [1]:
import numpy as np

In [12]:
def _hog_normalize_block(block, eps=1e-5):
    return block / (np.sum(np.abs(block)) + eps)

In [13]:
def _hog_channel_gradient(channel):
    g_row = np.empty(channel.shape, dtype=channel.dtype)
    g_row[0, :] = 0
    g_row[-1, :] = 0
    g_row[1:-1, :] = channel[2:, :] - channel[:-2, :]
    g_col = np.empty(channel.shape, dtype=channel.dtype)
    g_col[:, 0] = 0
    g_col[:, -1] = 0
    g_col[:, 1:-1] = channel[:, 2:] - channel[:, :-2]
    return g_row, g_col

In [24]:
def cell_hog(magnitude,orientation,orientation_start, orientation_end, cell_columns, cell_rows,\
             column_index, row_index, size_columns, size_rows, range_rows_start, range_rows_stop,\
             range_columns_start, range_columns_stop):
    
    total = 0.

    for cell_row in range(range_rows_start, range_rows_stop):
        cell_row_index = row_index + cell_row
        if (cell_row_index < 0 or cell_row_index >= size_rows):
            continue

        for cell_column in range(range_columns_start, range_columns_stop):
            cell_column_index = column_index + cell_column
            if (cell_column_index < 0 or cell_column_index >= size_columns
                    or orientation[cell_row_index, cell_column_index]
                    >= orientation_start
                    or orientation[cell_row_index, cell_column_index]
                    < orientation_end):
                continue

            total += magnitude[cell_row_index, cell_column_index]

    return total / (cell_rows * cell_columns)


def hog_histograms(gradient_columns,gradient_rows,cell_columns,cell_rows,size_columns, size_rows,\
                   number_of_cells_columns, number_of_cells_rows, number_of_orientations, orientation_histogram):

    magnitude = np.hypot(gradient_columns,gradient_rows)
    orientation = np.rad2deg(np.arctan2(gradient_rows, gradient_columns)) % 180

    r_0 = cell_rows // 2
    c_0 = cell_columns // 2
    cc = cell_rows * number_of_cells_rows
    cr = cell_columns * number_of_cells_columns
    range_rows_stop = int((cell_rows + 1) / 2)
    range_rows_start = int(-(cell_rows / 2))
    range_columns_stop = int((cell_columns + 1) / 2)
    range_columns_start = int(-(cell_columns / 2))
    number_of_orientations_per_180 = 180. / number_of_orientations

    for i in range(number_of_orientations):
        orientation_start = number_of_orientations_per_180 * (i + 1)
        orientation_end = number_of_orientations_per_180 * i
        c = c_0
        r = r_0
        r_i = 0
        c_i = 0
        while r < cc:
            c_i = 0
            c = c_0
            while c < cr:
                orientation_histogram[r_i, c_i, i] = \
                    cell_hog(magnitude, orientation,
                             orientation_start, orientation_end,
                             cell_columns, cell_rows, c, r,
                             size_columns, size_rows,
                             range_rows_start, range_rows_stop,
                             range_columns_start, range_columns_stop)
                c_i += 1
                c += cell_columns

            r_i += 1
            r += cell_rows

In [28]:
def hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3)):
    g_row, g_col = _hog_channel_gradient(image)

    s_row, s_col = image.shape[:2]
    c_row, c_col = pixels_per_cell
    b_row, b_col = cells_per_block

    n_cells_row = int(s_row // c_row)  # number of cells along row-axis
    n_cells_col = int(s_col // c_col)  # number of cells along col-axis

    # compute orientations integral images
    orientation_histogram = np.zeros((n_cells_row, n_cells_col, orientations),
                                     dtype=float)
    g_row = g_row.astype(float, copy=False)
    g_col = g_col.astype(float, copy=False)

    hog_histograms(g_col, g_row, c_col, c_row, s_col, s_row,
                                 n_cells_col, n_cells_row,
                                 orientations, orientation_histogram)

    # now compute the histogram for each cell
    hog_image = None

    n_blocks_row = (n_cells_row - b_row) + 1
    n_blocks_col = (n_cells_col - b_col) + 1
    normalized_blocks = np.zeros((n_blocks_row, n_blocks_col, b_row, b_col, orientations))

    for r in range(n_blocks_row):
        for c in range(n_blocks_col):
            block = orientation_histogram[r:r + b_row, c:c + b_col, :]
            normalized_blocks[r, c, :] = \
                _hog_normalize_block(block)

    
    normalized_blocks = normalized_blocks.ravel()

    return normalized_blocks

## test

In [29]:
import pandas as pd
from skimage.color import rgb2gray

In [10]:
Xtr = pd.read_csv('data/Xtr.csv',header=None,sep=',',usecols=range(3072))
Xtr_ = np.array(Xtr).reshape(5000, 3,32, 32)
X = Xtr_.swapaxes(1,2).swapaxes(2,3)

In [30]:
image = rgb2gray(X[2])

fd = hog(image)

In [31]:
fd

array([0.00923451, 0.01077985, 0.03700901, 0.01162998, 0.01093883,
       0.00273301, 0.        , 0.        , 0.00519795, 0.01387317,
       0.010983  , 0.00514529, 0.0174079 , 0.02069694, 0.00608613,
       0.00456594, 0.0038158 , 0.00411755, 0.00741287, 0.00922542,
       0.02696134, 0.00408821, 0.00908161, 0.00785695, 0.01368763,
       0.00726234, 0.00540549, 0.01169456, 0.01546761, 0.02891789,
       0.01181312, 0.01490952, 0.01716532, 0.01605114, 0.00368275,
       0.00611713, 0.01640066, 0.00981437, 0.01006756, 0.00749353,
       0.00719262, 0.00781372, 0.01563116, 0.02292827, 0.00484144,
       0.00615829, 0.00271257, 0.00604552, 0.00682485, 0.01681687,
       0.00436158, 0.01294323, 0.040519  , 0.02978122, 0.01697047,
       0.00569356, 0.00653326, 0.00745763, 0.02291483, 0.0113539 ,
       0.01776988, 0.01189137, 0.00528297, 0.00400462, 0.0034865 ,
       0.00393276, 0.00150685, 0.01958089, 0.02812965, 0.01450262,
       0.01761653, 0.00415439, 0.00726038, 0.03088928, 0.01262